In [2]:
import pandas as pd
import mygene
from gprofiler import GProfiler


ImportError: cannot import name 'GProfiler' from 'gprofiler' (unknown location)

In [4]:
# -------------------------------
# 1. Load and filter the predictions
# -------------------------------
# Assuming your CSV file is named "predictions.csv"
df = pd.read_csv("../results/emb_extraction_model_pred.csv")

# Filter for genes with Label 0 but predicted as 1 (false positives)
false_positives = df[(df['Label'] == 0) & (df['Pred'] == 1)]
print("Genes with Label 0 but predicted as 1:")
print(false_positives)

# -------------------------------
# 2. Retrieve gene information using MyGene.info
# -------------------------------
mg = mygene.MyGeneInfo()
gene_info = {}

for gene in false_positives['Gene']:
    # Query for gene details: symbol, name, summary, Gene Ontology terms, and UniProt links
    result = mg.query(gene, species='human', fields='symbol,name,summary,go,uniprot', size=1)
    if result['hits']:
        gene_info[gene] = result['hits'][0]
    else:
        gene_info[gene] = "No information found"

# Print out the gene information
print("\nBiological information for each false positive gene:")
for gene, info in gene_info.items():
    print(f"\nGene: {gene}")
    if isinstance(info, dict):
        print("Symbol:", info.get('symbol', 'N/A'))
        print("Name:", info.get('name', 'N/A'))
        print("Summary:", info.get('summary', 'N/A'))
        # Display GO annotations if available
        if 'go' in info:
            print("GO Annotations:")
            for category, terms in info['go'].items():
                print(f"  {category.upper()}:")
                # 'terms' may be a list or a single dict
                if isinstance(terms, list):
                    for term in terms:
                        print("    -", term.get('term', 'N/A'))
                elif isinstance(terms, dict):
                    print("    -", terms.get('term', 'N/A'))
        # Display UniProt info if available
        if 'uniprot' in info:
            print("UniProt:", info['uniprot'])
    else:
        print(info)
        

Input sequence provided is already in string format. No operation performed


Genes with Label 0 but predicted as 1:
            Gene  Label  Pred     Score
2           NOS1    0.0     1  0.851384
4          ABCC9    0.0     1  1.000000
30          FNTB    0.0     1  0.974066
75          CDK1    0.0     1  0.999710
76         CCNB1    0.0     1  0.994031
...          ...    ...   ...       ...
13564     ABCC11    0.0     1  0.999998
13583     ZNF582    0.0     1  0.999664
13595       FAAH    0.0     1  0.999997
13598   HLA-DRB1    0.0     1  1.000000
13612  KIAA1549L    0.0     1  1.000000

[954 rows x 4 columns]


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
Input sequen


Biological information for each false positive gene:

Gene: NOS1
Symbol: NOS1
Name: nitric oxide synthase 1
Summary: The protein encoded by this gene belongs to the family of nitric oxide synthases, which synthesize nitric oxide from L-arginine. Nitric oxide is a reactive free radical, which acts as a biologic mediator in several processes, including neurotransmission, and antimicrobial and antitumoral activities. In the brain and peripheral nervous system, nitric oxide displays many properties of a neurotransmitter, and has been implicated in neurotoxicity associated with stroke and neurodegenerative diseases, neural regulation of smooth muscle, including peristalsis, and penile erection. This protein is ubiquitously expressed, with high level of expression in skeletal muscle. Multiple transcript variants that differ in the 5' UTR have been described for this gene but the full-length nature of these transcripts is not known. Additionally, alternatively spliced transcript variants enc

In [ ]:

# -------------------------------
# 3. Functional enrichment analysis using g:Profiler
# -------------------------------
gp = GProfiler(return_dataframe=True)
genes_list = false_positives['Gene'].tolist()
enrichment_results = gp.profile(organism='hsapiens', query=genes_list)

print("\nEnrichment Analysis Results:")
print(enrichment_results.head(10))  # display the top 10 enriched terms